In [ ]:
# numpy for numerics, sympy for algebra, plotly and matplotlib for plots
import numpy as np
import cupy as cp
from sympy import *
from sympy.abc import x, t
from IPython.display import display, Math
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
import numpy as np
import os

plt.rcParams['figure.figsize'] = [10,6]
plt.rcParams.update({'font.size': 10})

In [ ]:
_nx = 180
_ny = 180
_PPU = 10
_mx = int(np.round(_nx/_PPU))
_my = int(np.round(_ny/(np.sqrt(3)*_PPU/2)))
_dx = (4*np.pi*_mx/np.sqrt(3))/_nx
_dy = (2*np.pi*_my)/_ny
_kx, _ky = np.meshgrid(cp.fft.fftfreq(_nx, d=_dx)*2*np.pi, cp.fft.fftfreq(_ny, d=_dy)*2*np.pi)
_k2 = _kx**2 + _ky**2

def GetEtaNoise():
    noisex = cp.random.normal(loc=0, scale=1, size=(_ny, _nx))
    noisex_fft = 1j*_kx*cp.fft.fft2(noisex)
    noisey = cp.random.normal(loc=0, scale=1, size=(_ny, _nx))
    noisey_fft = 1j*_ky*cp.fft.fft2(noisey)
    noise_fft = noisex_fft + noisey_fft

    # normalize
    noise_fft[0,0] = 0

    # high frequency cutoff
    noise_fft[_k2 > 1] = 0
    noise = cp.fft.ifft2(noise_fft).real
    return noise

In [ ]:
_noise = GetEtaNoise()

fig = go.Figure(data=go.Heatmap(z=_noise.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_nx*2 + 100,
    height=_ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

In [ ]:
fig = go.Figure(data=go.Heatmap(z=_k2.get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_nx*2 + 100,
    height=_ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

In [ ]:
_kt = cp.fft.fftfreq(30, d=1)
_kt

In [ ]:
_kt2 = _kt**2
_kt2

In [ ]:
_noiseT = cp.zeros((_ny, _nx, 30))
for i in range(30):
    _noiseT[:,:,i] = GetEtaNoise()

In [ ]:
_noiseT_hat = cp.fft.fft2(_noiseT)

In [ ]:
_noiseT_hat[:,:,_kt2 > 0.01] = 0
_noiseT = cp.fft.ifft2(_noiseT_hat).real

In [ ]:
_noiseT.shape

In [ ]:
fig = go.Figure(data=go.Heatmap(z=_noiseT[:,:,29].get(), colorscale='hot', colorbar_thickness=10))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_nx*2 + 100,
    height=_ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

In [ ]:
fig = go.Figure(data=go.Heatmap(z=_noiseT[:,:,22].get(), colorscale='hot', colorbar_thickness=10, zmin=-0.3, zmax=0.3))
fig.update_layout(
    yaxis=dict(scaleanchor="x"),
    width=_nx*2 + 100,
    height=_ny*2 + 100,
    margin=dict(l=0, r=0, t=0, b=0),
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    xaxis_visible=False,
    yaxis_visible=False,
    plot_bgcolor='rgba(0,0,0,0)'
)
fig.show()

In [ ]:
_noiseT.shape

In [ ]:
cp.array([GetEtaNoise()]).shape

In [ ]:

outputPath = f'noise_T30'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

for i in range(1000):
    # noise is (ny, nx, nt)
    # remove the first time step and add a new one
    _noiseT = cp.concatenate((_noiseT[:,:,1:], cp.array([GetEtaNoise()]).reshape(_ny,_nx,1)), axis=2)
    _noiseT.shape

    _noiseT_hat = cp.fft.fft2(_noiseT)
    _noiseT_hat[:,:,_kt2 > 0.004] = 0
    _noiseT = cp.fft.ifft2(_noiseT_hat).real

    fig = go.Figure(data=go.Heatmap(z=_noiseT[:,:,-1].get(), colorscale='hot', colorbar_thickness=10, zmin=-0.3, zmax=0.3))
    fig.update_layout(
        yaxis=dict(scaleanchor="x"),
        width=_nx*2 + 100,
        height=_ny*2 + 100,
        margin=dict(l=0, r=0, t=0, b=0),
        xaxis_showgrid=False,
        yaxis_showgrid=False,
        xaxis_zeroline=False,
        yaxis_zeroline=False,
        xaxis_visible=False,
        yaxis_visible=False,
        plot_bgcolor='rgba(0,0,0,0)'
    )
    fig.write_image(os.path.join(outputPath, f'noise{i:04}.png'))
    if i % 10 == 9:
        fig.show()

In [ ]:
outputPath = f'noise_T0'
if not os.path.exists(outputPath):
  os.mkdir(outputPath)

for i in range(1000):
    # noise is (ny, nx, nt)
    # remove the first time step and add a new one
    _noiseT = cp.concatenate((_noiseT[:,:,1:], cp.array([GetEtaNoise()]).reshape(_ny,_nx,1)), axis=2)
    _noiseT.shape

    _noiseT_hat = cp.fft.fft2(_noiseT)
    _noiseT_hat[:,:,_kt2 > 1] = 0
    _noiseT = cp.fft.ifft2(_noiseT_hat).real

    fig = go.Figure(data=go.Heatmap(z=_noiseT[:,:,-1].get(), colorscale='hot', colorbar_thickness=10, zmin=-0.6, zmax=0.6))
    fig.update_layout(
        yaxis=dict(scaleanchor="x"),
        width=_nx*2 + 100,
        height=_ny*2 + 100,
        margin=dict(l=0, r=0, t=0, b=0),
        xaxis_showgrid=False,
        yaxis_showgrid=False,
        xaxis_zeroline=False,
        yaxis_zeroline=False,
        xaxis_visible=False,
        yaxis_visible=False,
        plot_bgcolor='rgba(0,0,0,0)'
    )
    fig.write_image(os.path.join(outputPath, f'noise{i:04}.png'))
    if i % 10 == 9:
        fig.show()